In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
def get_transactions(address, limit=50):
    base_url = 'https://btcscan.org/api'
    txs_endpoint = f'/address/{address}/txs/chain'
    all_transactions = []
    last_seen_txid = ''

    while len(all_transactions) < limit:
        url = f"{base_url}{txs_endpoint}"
        if last_seen_txid:
            url += f'/{last_seen_txid}'

        response = requests.get(url)

        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break

        transactions = response.json()
        if not transactions:
            break

        all_transactions.extend(transactions[:limit - len(all_transactions)])  # Ensure only 'limit' transactions are collected

        last_seen_txid = transactions[-1]['txid']

        if len(transactions) < 25:  # Assuming 25 is the batch size returned by the API
            break

    return all_transactions





In [ ]:
def extract_addresses(transactions, target_address):
    involved_addresses = []
    transactions_data = []

    for tx in transactions:
        txid = tx['txid']
        tx_time = datetime.fromtimestamp(tx['status']['block_time'])  # Convert timestamp to datetime
        tx_addresses = set()
        direction = None

        # Check inputs
        for vin in tx['vin']:
            amount = vin['prevout']['value']
            vin_address = vin['prevout'].get('scriptpubkey_address')
            if vin_address == target_address:
                direction = "Outgoing"
            tx_addresses.add((vin_address, txid, direction, amount))

        # Check outputs
        for vout in tx['vout']:
            amount = vout['value']
            vout_address = vout.get('scriptpubkey_address')
            if vout_address == target_address:
                direction = "Incoming"
            tx_addresses.add((vout_address, txid, direction, amount))

        involved_addresses.extend(tx_addresses)

        # Store transaction data for CSV
        transactions_data.append({
            'txid': txid,
            'time': tx_time,
            'direction': direction,
            'amount': amount
        })

    return involved_addresses, transactions_data

In [3]:
def main():

    # Replace 'bc1q0kpk303cljyn62vgve34hcxlztn7zfmh0jgjtf' with the bitcoin address you want to analyze
    address = 'bc1q0kpk303cljyn62vgve34hcxlztn7zfmh0jgjtf'
    transactions = get_transactions(address)

    if transactions:
        involved_addresses, transactions_data = extract_addresses(transactions, address)
        unique_txids = set(tx['txid'] for tx in transactions)

        print(f"Total unique transactions found: {len(unique_txids)}")
        print("Addresses involved, their transaction IDs, direction, and amount:")

        for addr, txid, direction, amount in involved_addresses:
            print(f"Address: {addr}, Transaction ID: {txid}, Direction: {direction}, Amount: {amount}")

        # Save transactions to a CSV file
        df = pd.DataFrame(transactions_data)
        df.to_csv("btc_transactions.csv", index=False)

    else:
        print("No transactions found.")

if __name__ == "__main__":
    main()


Total unique transactions found: 50
Addresses involved, their transaction IDs, direction, and amount:
Address: bc1q5kx8976cz6k3dhlcyht0znh22gr754uygq8xhl, Transaction ID: 2cbde48199c3a30ca797197b25c7f9adab78fa182334dbbc0226cff2c8ea8ad8, Direction: Outgoing, Amount: 15000
Address: bc1q0kpk303cljyn62vgve34hcxlztn7zfmh0jgjtf, Transaction ID: 2cbde48199c3a30ca797197b25c7f9adab78fa182334dbbc0226cff2c8ea8ad8, Direction: Outgoing, Amount: 35037
Address: bc1q0kpk303cljyn62vgve34hcxlztn7zfmh0jgjtf, Transaction ID: 2cbde48199c3a30ca797197b25c7f9adab78fa182334dbbc0226cff2c8ea8ad8, Direction: Outgoing, Amount: 30180
Address: bc1q5kx8976cz6k3dhlcyht0znh22gr754uygq8xhl, Transaction ID: 2cbde48199c3a30ca797197b25c7f9adab78fa182334dbbc0226cff2c8ea8ad8, Direction: Outgoing, Amount: 31578
Address: bc1q5kx8976cz6k3dhlcyht0znh22gr754uygq8xhl, Transaction ID: 2cbde48199c3a30ca797197b25c7f9adab78fa182334dbbc0226cff2c8ea8ad8, Direction: Outgoing, Amount: 116041
Address: bc1q5kx8976cz6k3dhlcyht0znh22gr754uygq